# <center>Big Data For Engineers&ndash; Solutions</center>
## <center>Spring 2023 &ndash; Week 1 &ndash; ETH Zurich</center>

### Aims
- **After this exercise:** Understand the SQL language and its common query patterns.
- **Later in the semester:** Relate these language features and query patterns relative to other data shapes, technologies, and the languages designed to query them.



- **After this exercise:** Understand the 'table' data shape, normalization, and when they can (and should) be used.
- **Later in the semester:** Understand when you can (and should) throw all of this away!

### Prerequisites
In this exercise, you will brush-up the fundamental concepts of relational databases and SQL. If you haven't taken an introductory databases course (or want to refresh your knowledge) we recommend you to read the following:

Garcia-Molina, Ullman, Widom: Database Systems: The Complete Book. Pearson, 2. Edition, 2008. (Chapters 1, 2, 3, and 6) [Available in the ETH Library] [[Online]](https://ebookcentral.proquest.com/lib/ethz/detail.action?pq-origsite=primo&docID=5832965) [[Selected solutions]](http://infolab.stanford.edu/~ullman/dscbsols/sols.html)

### Database Set-up
Unlike last week's exercise, the dataset for this exercise might take a little bit longer to download and initialize. Please wait for the message `PostgreSQL init process complete; ready for start up` before proceeding!

As before, we set up our connection to the database and enable use of `%sql` and `%%sql`.

In [ ]:
server='postgres'
user='postgres'
password='BigData1'
database='discogs'
connection_string=f'postgresql://{user}:{password}@{server}:5432/{database}'

In [ ]:
%reload_ext sql
%sql $connection_string

In [ ]:
%%sql
SELECT version();

## Exercise 1: Explore the dataset
We want to first understand the dataset a bit better. You will find some queries below to help you explore the schema. In the process, consider the following questions:

1. Which concepts are modelled in the dataset and how do they relate to each other?
2. The data is stored as tables. Why was this shape chosen and why not the other shapes?
3. In which normal forms are the corresponding relations?
4. What are the efficiency trade-offs from adding an `artist_id` and `artist_name` directly to the `releases` table? Hints:
   - What are some typical queries that would benefit from this change?
   - How often do we need to update artists?
5. What potential problems could result from adding this redundancy?

### Where we got the data from
- [Discogs](https://www.discogs.com/)
- [Discogs XML data dumps](http://data.discogs.com/)
- [Download the dataset](https://cloud.inf.ethz.ch/s/4bZWo4TjeXgCNz5) (only necessary if you don't want to use Docker, see `postgres-init.sh` to see how to import it)

### List tables
The following query retrieves a list of tables in the database from a system table describing the current database.

In [ ]:
%%sql 
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

### List attributes/columns
The following query retrieves a list of columns from the tables in the database.

In [ ]:
%%sql 
SELECT table_name, column_name, data_type, is_nullable, ordinal_position
FROM information_schema.columns
WHERE table_schema = 'public'
AND table_name NOT LIKE 'pg_%'
ORDER BY table_name, ordinal_position;

### Exercise 1: Solution
1. Which concepts are modelled in the dataset and how do they relate to each other?

   The dataset contains data from `discogs` and models musical artists, their physical releases, and the tracks on those releases.

2. The data is stored as tables. Why was this shape chosen and why not the other shapes?
   - The number of concepts is limited, fixed, and well-defined.
   - The same is true for attributes of these concepts.
   - Attributes come from a well-defined domains with a fixed semantic (such as dates, e-mail addresses, ...).
   - Instances of concepts are in relation with each other, which may or may not be required to exist.
   - In short: we can define a schema, which the rest of the application (the website) relies on.


3. In which normal forms are the corresponding relations?

   - All relations are atomic so they are in 1NF.
   - `artists` and `releases` both have a singular ID primary key, so they at least 2NF. Upon inspection, there are no transitive dependencies, so they are in 3NF.
   - `released_by` does not contain non-key attributes, so it is in 3NF.
   - `tracks` has a foreign key constraint but no primary key constraint (although this is not captured by normal forms). The reason is likely because the data is crowd-sourced and archival, and there could be duplicate entries in the database.


4. What are the efficiency trade-offs from adding an `artist_id` and `artist_name` directly to the `releases` table? Hints:

   - What are some typical queries that would benefit from this change?
   - How often do we need to update artists?
   
   A very typical query is finding all the releases by an artist's name. If we redundantly store the artist's ID and name in `releases`, the aforementioned query now only requires a `SELECT` instead of an expensive `JOIN`. As releases can be released by multiple artists, we could also extend this idea and break 1NF by storing a list of artist names instead. The tradeoff here is that every time we have to update an artist's name we have to find and update all releases by that artist. Considering that reads are very common and changes are rare, the option of denormalizing becomes relatively more attractive.

5. What potential problems could result from denormalizing the data?

   - Insert anomaly: we could insert a release with a fake or incorrect artist name.
   - Update anomaly: if we update an artist's name but not all of the records in `releases`, we introduce an inconsistency into the dataset.

## Exercise 2: SQL warm-up
Let us begin with several SQL queries to ease us back into the language.

1. Retrieve all artists with the name of 'Radiohead'.

In [ ]:
%%sql
SELECT * FROM artists WHERE name = 'Radiohead';

2. List the titles of all releases by that artist in alphabetical order.

In [ ]:
%%sql
SELECT DISTINCT releases.title
FROM artists
JOIN released_by USING(artist_id)
JOIN releases USING(release_id)
WHERE artists.name = 'Radiohead'
ORDER BY releases.title;

3. List the titles of all releases by that artist that contain less than 5 tracks.

In [ ]:
%%sql
SELECT releases.title, COUNT(*) AS num_tracks
FROM artists
JOIN released_by USING(artist_id)
JOIN releases USING(release_id)
JOIN tracks USING(release_id)
WHERE artists.name = 'Radiohead'
GROUP BY releases.release_id, releases.title
HAVING COUNT(*) < 5;

4. What are the top 10 artists with the most releases?

In [ ]:
%%sql
SELECT artists.artist_id, artists.name, COUNT(*) AS num_releases
FROM artists
JOIN released_by USING(artist_id)
JOIN releases USING(release_id)
GROUP BY artists.artist_id, artists.name
ORDER BY num_releases DESC
LIMIT 10;

5. How many artists have more releases than the average number of releases per artists (who have at least 1 release)

In [ ]:
%%sql
WITH releases_per_artist AS (
    SELECT artists.artist_id, artists.name, COUNT(*) AS release_count
    FROM artists
    JOIN released_by USING(artist_id)
    JOIN releases USING(release_id)
    GROUP BY artists.artist_id 
),
average_release_count AS (
    SELECT AVG(release_count)
    FROM releases_per_artist
)
SELECT COUNT(*)
FROM releases_per_artist
WHERE releases_per_artist.release_count > (SELECT * FROM average_release_count);

6. What are the names and IDs of the artists that have both a release with the genre 'Pop' *and* a release with the genre 'Classical'? Give a query that uses `INTERSECT` and one that uses `EXISTS`.

In [ ]:
%%sql
SELECT artists.artist_id, artists.name
FROM artists
JOIN released_by USING(artist_id)
JOIN releases USING(release_id)
WHERE releases.genre = 'Pop'
INTERSECT
SELECT artists.artist_id, artists.name
FROM artists
JOIN released_by USING(artist_id)
JOIN releases USING(release_id)
WHERE releases.genre = 'Classical'

In [ ]:
%%sql
SELECT a1.artist_id, a1.name
FROM artists a1
WHERE EXISTS (
    SELECT *
    FROM artists a2
    JOIN released_by USING(artist_id)
    JOIN releases USING(release_id)
    WHERE releases.genre = 'Pop'
    AND a1.artist_id = a2.artist_id
) AND EXISTS (
    SELECT *
    FROM artists a2
    JOIN released_by USING(artist_id)
    JOIN releases USING(release_id)
    WHERE releases.genre = 'Classical'
    AND a1.artist_id = a2.artist_id
)

## Exercise 3: Impact of release genre on average track duration and track count
For this exercise, we want to find out how average track duration and track count varies across genres.

To start, write a query which finds all of the distinct genres:

In [ ]:
%%sql
SELECT DISTINCT genre
FROM releases;

Take a guess as to which genre has:
1. The highest average track count?
2. The lowest average track count?
3. The longest average track duration?
4. The shortest average track duration?

Next, write a query to calculate the average track count per genre:

In [ ]:
%%sql 
WITH tracks_per_release AS (
    SELECT releases.genre, COUNT(*) AS track_count
    FROM releases 
    JOIN tracks USING(release_id)
    GROUP BY releases.release_id, releases.genre
)
SELECT tracks_per_release.genre, AVG(tracks_per_release.track_count) AS avg_track_count
FROM tracks_per_release
GROUP BY tracks_per_release.genre
ORDER BY avg_track_count ASC;

Write a query to calculate the average duration per genre. Your result should have two attributes: `genre` and `avg_duration`.

In [ ]:
%%sql
SELECT releases.genre, AVG(tracks.duration) AS avg_duration
FROM releases JOIN tracks USING(release_id)
GROUP BY releases.genre
ORDER BY avg_duration ASC;

Did the results match what you expected? Copy your query into the following python script to plot the result.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Store the result of the query in a Python object (add your query here!)
result = %sql   SELECT releases.genre, AVG(tracks.duration) AS avg_duration \
                FROM releases JOIN tracks USING(release_id) \
                GROUP BY releases.genre \
                ORDER BY avg_duration ASC; \

# Convert the result to a Pandas data frame
df = result.DataFrame()

# Extract x and y values for a plot
x = df['genre'].tolist()
y = df['avg_duration'].tolist()

# Print them just for debugging
print(x)
print(y)

# Plot the distribution of scores
fig = plt.figure(figsize =(14, 7))
plt.barh(x, y, align='center')
plt.xlabel('Average Duration (s)')
plt.ylabel('Genre')

## Exercise 4: Discuss query patterns and language features of SQL
1. What patterns did you use in many of the queries above?

2. Do you remember the theory behind them?

3. What makes SQL a declarative language and what advantages does that have?

4. What makes SQL a functional language and what advantages does that have?

### Exercise 4: Solution
1. Most queries consist of the following basic operations. They will reoccur throughout the whole semester. Watch out for them!
  * **Select**: select a subset of the rows/data records/items.
  * **Project**: select a subset of the properties/ attributes/columns.
  * **Join**: bring two datasets together based on a common attribute.
  * **Group**: divide the items/ rows/records into groups and summarize each group with a single value.
  * **Order**: order the items according to some criteria.
  
2. Relational algebra operators formalize most of this (grouping is technically not part of the algebra).
  
3. We only describe *what* we want, not how this should be computed. We *declare* what our intent is. This shifts the implementation effort from the programmer to the database system. The hope is that the system has more information at hand, such as data size, data distribution, information about the hardware, in order to choose the best way to compute the result. This results into efficient computation with little effort from the programmer.
  
4. SQL is functional because results of a query can be used as input of another query, either in form of tables or in form of scalars. This makes SQL expressive.